In [1]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_csv('/Users/mac/Desktop/Project/playground-series-s3e19/train.csv')
test = pd.read_csv('/Users/mac/Desktop/Project/playground-series-s3e19/test.csv')

In [3]:
data.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [4]:
data.isnull().sum()

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [5]:
print('Value_counts', data['date'].value_counts())

Value_counts 2017-01-01    75
2020-05-11    75
2020-05-09    75
2020-05-08    75
2020-05-07    75
              ..
2018-08-29    75
2018-08-28    75
2018-08-27    75
2018-08-26    75
2021-12-31    75
Name: date, Length: 1826, dtype: int64


In [6]:
print('Value_counts', data['country'].value_counts())

Value_counts Argentina    27390
Canada       27390
Estonia      27390
Japan        27390
Spain        27390
Name: country, dtype: int64


In [7]:
print('Value_counts', data['store'].value_counts())

Value_counts Kaggle Learn    45650
Kaggle Store    45650
Kagglazon       45650
Name: store, dtype: int64


In [8]:
print('Value_counts', data['product'].value_counts())

Value_counts Using LLMs to Improve Your Coding                 27390
Using LLMs to Train More LLMs                     27390
Using LLMs to Win Friends and Influence People    27390
Using LLMs to Win More Kaggle Competitions        27390
Using LLMs to Write Better                        27390
Name: product, dtype: int64


In [9]:
from sklearn.preprocessing import LabelEncoder
data['country'] = LabelEncoder().fit(data['country']).transform(data['country'])
data['store'] = LabelEncoder().fit(data['store']).transform(data['store'])
data['product'] = LabelEncoder().fit(data['product']).transform(data['product'])

In [10]:
data.drop(columns = ['date'], inplace = True)

In [11]:
data.head()

,id,country,store,product,num_sold
0,0,0,1,0,63
1,1,0,1,1,66
2,2,0,1,2,9
3,3,0,1,3,59
4,4,0,1,4,49


In [12]:
from sklearn.model_selection import train_test_split
X = data.drop(columns = ['num_sold'])
Y = data['num_sold']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
X_train_nor = StandardScaler().fit(x_train).transform(x_train)
X_test_nor = StandardScaler().fit(x_test).transform(x_test)

In [14]:
print('Train shape', X_train_nor.shape, y_train.shape)
print('Test shape', X_test_nor.shape, y_test.shape)

Train shape (95865, 4) (95865,)
Test shape (41085, 4) (41085,)


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor

search = GridSearchCV(KNeighborsRegressor(), {
    'n_neighbors' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['ball_tree', 'kd_tree', 'brute', 'auto'], 
})
search.fit(X_train_nor, y_train)
search.cv_results_


{'mean_fit_time': array([0.11750236, 0.07810144, 0.05611057, 0.0542232 , 0.05791416,
        0.11138539, 0.10802016, 0.07389679, 0.1103159 , 0.1149044 ,
        0.0899281 , 0.12685466, 0.06571331, 0.07031651, 0.05565405,
        0.1402493 , 0.10790071, 0.06235905, 0.10221081, 0.10589843,
        0.1204504 , 0.11684828, 0.05467   , 0.09596066, 0.08090019,
        0.0523243 , 0.05930181, 0.05348887, 0.05374503, 0.07147703,
        0.14573779, 0.09560165, 0.09830375, 0.09244151, 0.15382104,
        0.07799625, 0.13020205, 0.0628016 , 0.06220131, 0.06636968,
        0.00718975, 0.00709572, 0.00666304, 0.00706334, 0.00685368,
        0.0066987 , 0.0067482 , 0.00674715, 0.00755987, 0.00676317,
        0.00671873, 0.00717235, 0.00866132, 0.00736632, 0.0072917 ,
        0.00775566, 0.00841894, 0.01139975, 0.00659218, 0.0065866 ,
        0.05142097, 0.04751635, 0.04730277, 0.04688292, 0.04673438,
        0.04770141, 0.04612498, 0.0462564 , 0.04642282, 0.04534073,
        0.04822631, 0.04723105,

In [16]:
df = pd.DataFrame(search.cv_results_)
df[['param_n_neighbors', 'param_weights', 'param_algorithm', 'mean_test_score']].sort_values(by = 'mean_test_score', ascending = False)

,param_n_neighbors,param_weights,param_algorithm,mean_test_score
79,10,distance,auto,0.982788
39,10,distance,kd_tree,0.982788
19,10,distance,ball_tree,0.982788
59,10,distance,brute,0.982784
17,9,distance,ball_tree,0.982725
...,...,...,...,...
20,1,uniform,kd_tree,0.975525
0,1,uniform,ball_tree,0.975522
1,1,distance,ball_tree,0.975522
41,1,distance,brute,0.975313


In [17]:
print('Best estimator', search.best_estimator_)
print('Best parameter', search.best_params_)
print('Best score', search.best_score_)

Best estimator KNeighborsRegressor(algorithm='kd_tree', n_neighbors=10, weights='distance')
Best parameter {'algorithm': 'kd_tree', 'n_neighbors': 10, 'weights': 'distance'}
Best score 0.9827880791552882


In [19]:
neighbor = KNeighborsRegressor(algorithm = 'kd_tree', n_neighbors = 10, weights = 'distance')
neighbor.fit(X_train_nor, y_train)
predict = neighbor.predict(X_test_nor)
print('MSE', mean_squared_error(y_test, predict))
print('Rsquare', r2_score(y_test, predict))

MSE 700.8852371439051
Rsquare 0.9791801098789452


In [34]:
test.head()

,id,date,country,store,product
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better


In [35]:
test.isnull().sum()

id         0
date       0
country    0
store      0
product    0
dtype: int64

In [36]:
test['country'] = LabelEncoder().fit(test['country']).transform(test['country'])
test['store'] = LabelEncoder().fit(test['store']).transform(test['store'])
test['product'] = LabelEncoder().fit(test['product']).transform(test['product'])

In [37]:
test.drop(columns = ['date'], inplace = True)

In [24]:
test.head()

,id,country,store,product
0,136950,0,1,0
1,136951,0,1,1
2,136952,0,1,2
3,136953,0,1,3
4,136954,0,1,4


In [38]:
x_train_nor = StandardScaler().fit(test).transform(test)
print('x_train_nor.shape', x_train_nor.shape)

x_train_nor.shape (27375, 4)


In [39]:
y = neighbor.predict(x_train_nor)
y.reshape(27375, 1)

array([[ 43.04577819],
       [ 48.71757188],
       [  7.01227603],
       ...,
       [103.3479717 ],
       [580.20546583],
       [460.93164018]])

In [40]:
submission = pd.DataFrame(y)
test = pd.concat([test, submission], axis = 1)
test.drop(columns = ['country', 'store', 'product'], inplace = True)
test.rename(columns = {0 : 'num_sold'}, inplace = True)
test.head()

,id,num_sold
0,136950,43.045778
1,136951,48.717572
2,136952,7.012276
3,136953,39.235046
4,136954,37.521814


In [42]:
test.to_csv('/Users/mac/Desktop/Submission_KNR.csv', index = False)